Pip install opencv

In [10]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ASUS\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


## Importing Needed Libraries

In [11]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from numba import jit, cuda

## First storing data for preprocesssing

In [12]:
data = []
labels = []
classes = 43
cur_path = os.getcwd()

#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

In [13]:
# from keras.preprocessing import image

# for j in range(43,45):
#     path1 = os.path.join(cur_path,'train',str(j))
#     directory = os.listdir(path1)
#     for i in directory:
#         try:
#             test_image = image.load_img(f"{path1}\{i}", target_size = (30, 30))
#             test_image = image.img_to_array(test_image)
#             data.append(list(test_image))
#             labels.append(j)
#         except:
#             print("error")


## Further Preprocessing 

In [17]:

# Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)

# # print(data)
print(data.shape, labels.shape)
#Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# #Converting the labels into one hot encoding
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

(39209, 30, 30, 3) (39209,)


# Building the model

In [18]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(43, activation='softmax'))


##Compilation of Model

In [19]:
# #Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# @jit(target ="cuda")
# epoch = 50
#more accuracy increase the epochs size
history = model.fit(X_train, y_train, batch_size=64, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
491/491 [==============================] - 58s 116ms/step - loss: 1.2574 - accuracy: 0.6879 - val_loss: 0.1362 - val_accuracy: 0.9716
Epoch 2/20
491/491 [==============================] - 60s 123ms/step - loss: 0.1822 - accuracy: 0.9538 - val_loss: 0.0571 - val_accuracy: 0.9862
Epoch 3/20
491/491 [==============================] - 61s 124ms/step - loss: 0.1081 - accuracy: 0.9723 - val_loss: 0.0487 - val_accuracy: 0.9893
Epoch 4/20
491/491 [==============================] - 62s 127ms/step - loss: 0.0897 - accuracy: 0.9767 - val_loss: 0.0306 - val_accuracy: 0.9923
Epoch 5/20
491/491 [==============================] - 63s 128ms/step - loss: 0.0714 - accuracy: 0.9828 - val_loss: 0.0278 - val_accuracy: 0.9940
Epoch 6/20
491/491 [==============================] - 71s 145ms/step - loss: 0.0596 - accuracy: 0.9857 - val_loss: 0.0315 - val_accuracy: 0.9922
Epoch 7/20
491/491 [==============================] - 68s 138ms/step - loss: 0.0736 - accuracy: 0.9816 - val_loss: 0.0400 - val_ac

## SAving the Model for further Use

In [ ]:
model.save("Alert_classifier.h5")